In [1]:
from sas7bdat import SAS7BDAT
import pandas as pd
from IPython.display import display
import gc
import pyreadstat

In [2]:
# kyrbs_file_path = 'c:\\data\\education\\kyrbs2024_sas\\kyrbs\\kyrbs2024.sas7bdat'
# pop_file_path = 'c:\\data\\education\\kyrbs2024_sas\\kyrbs\\pop24.sas7bdat'

# enc = 'euc-kr'
# df1, meta1 = pyreadstat.read_sas7bdat(kyrbs_file_path, encoding=enc)
# df2, meta2 = pyreadstat.read_sas7bdat(pop_file_path, encoding=enc)

# display(df1.head())
# display(df2.head())

# df1.to_csv('c:\\data\\education\\kyrbs2024_sas\\kyrbs\\kyrbs2024.csv', encoding='cp949', index=False)
# df2.to_csv('c:\\data\\education\\kyrbs2024_sas\\kyrbs\\pop24.csv', encoding='cp949', index=False)

# del df1
# del df2
# gc.collect()

In [3]:
kyrbs = pd.read_csv('c:\\data\\education\\kyrbs2024_sas\\kyrbs\\kyrbs2024.csv', encoding='cp949')
pop = pd.read_csv('c:\\data\\education\\kyrbs2024_sas\\kyrbs\\pop24.csv', encoding='cp949')

display(kyrbs.head(3))
display(pop.head(3))

,OBS,mod_d,YEAR,CITY,CTYPE,CTYPE_SD,MH,SCHOOL,STYPE,STRATA,...,E_LT_F,E_LT_SF,E_LT_M,E_LT_SM,E_EDU_F,E_KRN_F,E_BORN_F,E_EDU_M,E_KRN_M,E_BORN_M
0,A100001,2024.11.26.,2024.0,서울,대도시,대도시,중학교,중학교,남녀공학,2024_64,...,1.0,9999.0,1.0,9999.0,3.0,1.0,9999.0,3.0,1.0,9999.0
1,A100002,2024.11.26.,2024.0,서울,대도시,대도시,중학교,중학교,남녀공학,2024_64,...,1.0,9999.0,1.0,9999.0,3.0,1.0,9999.0,3.0,1.0,9999.0
2,A100003,2024.11.26.,2024.0,서울,대도시,대도시,중학교,중학교,남녀공학,2024_64,...,1.0,9999.0,1.0,9999.0,3.0,1.0,9999.0,3.0,1.0,9999.0


,db_y,_total_,STRATA
0,2024.0,43.0,2024_1
1,2024.0,57.0,2024_10
2,2024.0,35.0,2024_100


In [4]:
missing_cols = kyrbs.columns[kyrbs.isna().any()].tolist()
print(missing_cols)
display(kyrbs[missing_cols].isna().sum())

['SCHOOL', 'PA_SWD_S', 'PA_SWD_N', 'PA_SWK_S', 'PA_SWK_N', 'HT', 'WT', 'M_SLP_HR', 'M_SLP_MM', 'M_WK_HR', 'M_WK_MM', 'AC_FAGE', 'TC_FAGE', 'TC_DAGE', 'TC_EC_FAGE', 'TC_EC_DAGE', 'TC_HTP_FAGE', 'TC_HTP_DAGE', 'S_FAGE', 'INT_SPWD_TM', 'INT_SPWK_TM', 'AGE', 'AGE_M', 'E_S_RCRD', 'E_SES', 'E_RES']


SCHOOL          441
PA_SWD_S        967
PA_SWD_N        967
PA_SWK_S       1004
PA_SWK_N       1004
HT             1480
WT             1480
M_SLP_HR       4140
M_SLP_MM       4140
M_WK_HR        4140
M_WK_MM        4140
AC_FAGE          59
TC_FAGE          87
TC_DAGE          32
TC_EC_FAGE       30
TC_EC_DAGE       23
TC_HTP_FAGE      32
TC_HTP_DAGE      15
S_FAGE          155
INT_SPWD_TM    1821
INT_SPWK_TM    1855
AGE              49
AGE_M            49
E_S_RCRD          3
E_SES             5
E_RES             7
dtype: int64

In [5]:
kyrbs['disturbed'] = kyrbs[[f'M_GAD_{i}' for i in range(1, 8)]].apply(lambda x: 1 if (x >= 3).any() else 0, axis=1) # 어떤것에 자주 방해 받는지
kyrbs['disturbed'].value_counts()

disturbed
0    35952
1    18701
Name: count, dtype: int64

In [6]:
kyrbs['violent'] = kyrbs['V_TRT'].apply(lambda x: 0 if x == 1 else (2 if x == 7 else 1)) # 폭력 경험 - 0 없음, 1 있음, 2 자주(6회 이상)
kyrbs['violent'].value_counts()

violent
0    53249
1     1246
2      158
Name: count, dtype: int64

In [7]:
kyrbs['haveparent'] = kyrbs['E_RES'].apply(lambda x: 1 if x==1 else 0)
kyrbs['haveparent'].value_counts()

haveparent
1    51961
0     2692
Name: count, dtype: int64

In [8]:
kyrbs['livefather'] = kyrbs[[i for i in ['E_LT_F', 'E_LT_SF']]].apply(lambda x: 1 if (x == 1).any() else 0, axis=1)
kyrbs['livemother'] = kyrbs[[i for i in ['E_LT_M', 'E_LT_SM']]].apply(lambda x: 1 if (x == 1).any() else 0, axis=1)
kyrbs['remarriaged'] = kyrbs[['E_LT_SF', 'E_LT_SM']].apply(lambda x: 1 if (x == 1).any() else 0, axis=1)

In [9]:
display(kyrbs[['livefather', 'livemother', 'remarriaged']].head(3))

,livefather,livemother,remarriaged
0,1,1,0
1,1,1,0
2,1,1,0


In [10]:
kyrbs.isna().sum()

OBS            0
mod_d          0
YEAR           0
CITY           0
CTYPE          0
              ..
violent        0
haveparent     0
livefather     0
livemother     0
remarriaged    0
Length: 200, dtype: int64

In [11]:
kyrbs.columns

Index(['OBS', 'mod_d', 'YEAR', 'CITY', 'CTYPE', 'CTYPE_SD', 'MH', 'SCHOOL',
       'STYPE', 'STRATA',
       ...
       'E_BORN_F', 'E_EDU_M', 'E_KRN_M', 'E_BORN_M', 'disturbed', 'violent',
       'haveparent', 'livefather', 'livemother', 'remarriaged'],
      dtype='object', length=200)

In [12]:
usecols = ['MH', 'E_S_RCRD', 'GRADE', 'M_STR', 'M_SLP_EN', 'M_SUI_CON', 'M_SUI_ATT', 'M_LON', 'disturbed', 'violent', 'haveparent', 'livefather', 'livemother', 'remarriaged']
kyrbs = kyrbs[usecols]
kyrbs

,MH,E_S_RCRD,GRADE,M_STR,M_SLP_EN,M_SUI_CON,M_SUI_ATT,M_LON,disturbed,violent,haveparent,livefather,livemother,remarriaged
0,중학교,1.0,1.0,4.0,1.0,1.0,1.0,2.0,0,0,1,1,1,0
1,중학교,4.0,1.0,3.0,3.0,1.0,1.0,2.0,0,0,1,1,1,0
2,중학교,1.0,1.0,2.0,1.0,1.0,1.0,3.0,0,0,1,1,1,0
3,중학교,2.0,1.0,1.0,5.0,2.0,2.0,5.0,1,0,1,0,1,0
4,중학교,2.0,1.0,3.0,2.0,1.0,1.0,3.0,0,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54648,고등학교,2.0,6.0,3.0,5.0,1.0,1.0,3.0,0,0,1,1,1,0
54649,고등학교,4.0,6.0,2.0,3.0,1.0,1.0,3.0,0,0,1,1,1,0
54650,고등학교,2.0,6.0,5.0,4.0,1.0,1.0,2.0,0,0,1,1,1,0
54651,고등학교,3.0,6.0,4.0,4.0,1.0,1.0,4.0,0,0,1,1,1,0


In [13]:
print(kyrbs['E_S_RCRD'].unique())
nan_rows = kyrbs[kyrbs['E_S_RCRD'].isna()]
nan_rows

[ 1.  4.  2.  3.  5. nan]


,MH,E_S_RCRD,GRADE,M_STR,M_SLP_EN,M_SUI_CON,M_SUI_ATT,M_LON,disturbed,violent,haveparent,livefather,livemother,remarriaged
23194,고등학교,NaN,5.0,2.0,5.0,1.0,1.0,3.0,0,0,0,0,0,0
36441,중학교,NaN,2.0,4.0,2.0,1.0,1.0,1.0,0,0,0,0,0,0
42141,고등학교,NaN,5.0,3.0,3.0,1.0,1.0,3.0,0,0,0,0,0,0


In [14]:
# print(kyrbs['E_SES'].unique())
# nan_rows = kyrbs[kyrbs['E_SES'].isna()]
# nan_rows

In [ ]:
# print(kyrbs['SCHOOL'].unique())
# nan_rows = kyrbs[kyrbs['SCHOOL'].isna()]
# nan_rows

In [16]:
kyrbs = kyrbs.dropna()
kyrbs

,MH,E_S_RCRD,GRADE,M_STR,M_SLP_EN,M_SUI_CON,M_SUI_ATT,M_LON,disturbed,violent,haveparent,livefather,livemother,remarriaged
0,중학교,1.0,1.0,4.0,1.0,1.0,1.0,2.0,0,0,1,1,1,0
1,중학교,4.0,1.0,3.0,3.0,1.0,1.0,2.0,0,0,1,1,1,0
2,중학교,1.0,1.0,2.0,1.0,1.0,1.0,3.0,0,0,1,1,1,0
3,중학교,2.0,1.0,1.0,5.0,2.0,2.0,5.0,1,0,1,0,1,0
4,중학교,2.0,1.0,3.0,2.0,1.0,1.0,3.0,0,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54648,고등학교,2.0,6.0,3.0,5.0,1.0,1.0,3.0,0,0,1,1,1,0
54649,고등학교,4.0,6.0,2.0,3.0,1.0,1.0,3.0,0,0,1,1,1,0
54650,고등학교,2.0,6.0,5.0,4.0,1.0,1.0,2.0,0,0,1,1,1,0
54651,고등학교,3.0,6.0,4.0,4.0,1.0,1.0,4.0,0,0,1,1,1,0


In [17]:
# 숫자형 변수
numeric_cols = ['M_STR', 'M_SLP_EN', 'M_SUI_CON', 'M_SUI_ATT', 'M_LON', 'E_S_RCRD']
for col in numeric_cols:
    kyrbs[col] = pd.to_numeric(kyrbs[col], errors='coerce')

# 범주형 변수 
categorical_cols = ['GRADE', 'disturbed', 'violent', 'haveparent', 'livefather', 'livemother', 'remarriaged']
for col in categorical_cols:
    kyrbs[col] = kyrbs[col].astype("category")

C:\Users\User\AppData\Local\Temp\ipykernel_19508\1393626140.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kyrbs[col] = pd.to_numeric(kyrbs[col], errors='coerce')
C:\Users\User\AppData\Local\Temp\ipykernel_19508\1393626140.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kyrbs[col] = kyrbs[col].astype("category")


In [18]:
mid = kyrbs[kyrbs['MH'] == '중학교']
high = kyrbs[kyrbs['MH'] == '고등학교']

len(mid), len(high)

(29086, 25564)

In [19]:
mid.columns

Index(['MH', 'E_S_RCRD', 'GRADE', 'M_STR', 'M_SLP_EN', 'M_SUI_CON',
       'M_SUI_ATT', 'M_LON', 'disturbed', 'violent', 'haveparent',
       'livefather', 'livemother', 'remarriaged'],
      dtype='object')

In [20]:
import pandas as pd
from semopy import Model, Optimizer, calc_stats

In [25]:
model_desc = """
School_Environment =~ E_S_RCRD
Stress_Causes =~ M_STR + M_SLP_EN + M_LON + disturbed
Suicidal_Factors =~ M_SUI_CON + M_SUI_ATT
Family_Environment =~ haveparent + livefather + livemother + remarriaged
Violence_Experience =~ violent

Stress_Causes ~ School_Environment + Family_Environment + Violence_Experience
Suicidal_Factors ~ Stress_Causes

M_SUI_ATT ~~ M_SUI_CON + M_STR
"""

# 구조 모델 피팅
m = Model(model_desc)
# 로버스트 추정 또는 부트스트랩 옵션을 추가할 수 있습니다.
m.fit(high)

# 구조 모델 결과 확인
stats = calc_stats(m)

print("적합도 지표:\n", stats.T)


적합도 지표:
                       Value
DoF               45.000000
DoF Baseline      66.000000
chi2            1965.979373
chi2 p-value       0.000000
chi2 Baseline  39326.569186
CFI                0.951071
GFI                0.950009
AGFI               0.926680
NFI                0.950009
TLI                0.928238
RMSEA              0.040865
AIC               65.846192
BIC              334.761224
LogLik             0.076904


In [26]:
estimates = m.inspect()
print(estimates)

                   lval  op                 rval  Estimate  Std. Err  \
0         Stress_Causes   ~   School_Environment -0.104770  0.013477   
1         Stress_Causes   ~   Family_Environment -0.661886  0.098206   
2         Stress_Causes   ~  Violence_Experience -0.700775  0.030771   
3      Suicidal_Factors   ~        Stress_Causes -0.217484  0.003734   
4              E_S_RCRD   ~   School_Environment  1.000000         -   
5                 M_STR   ~        Stress_Causes  1.000000         -   
6              M_SLP_EN   ~        Stress_Causes -0.588571  0.012221   
7                 M_LON   ~        Stress_Causes -1.079053  0.013924   
8             disturbed   ~        Stress_Causes -0.459069  0.006014   
9             M_SUI_CON   ~     Suicidal_Factors  1.000000         -   
10            M_SUI_ATT   ~     Suicidal_Factors  0.220143  0.008076   
11           haveparent   ~   Family_Environment  1.000000         -   
12           livefather   ~   Family_Environment  6.086794  0.24

In [27]:
from semopy import semplot, report
semplot(m, './high.png')
report(m, './report')